In [2]:
#imports
import spatialdata
import dask_image.imread
import dask.array as da
import napari_spatialdata

import sys, os
sys.path.append(os.path.abspath("../functions/"))
import anndata_utils

for package in [spatialdata, napari_spatialdata]:
    print(f"{package.__name__}: {package.__version__}")

/Users/jnimoca/Jose_BI/1_Pipelines/openDVP/.pixi/envs/default/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


spatialdata: 0.2.3
napari_spatialdata: 0.5.3


In [1]:
#paths
path_to_image = "../data/TNBC_subset/TD_15_TNBC_subset.ome.tif"
path_to_mask = "../data/TNBC_subset/mask.tif"
path_to_quant = "../data/TNBC_subset/quantification.csv" 

In [3]:
#create sdata
sdata = spatialdata.SpatialData()

In [4]:
#parse mIF image
image = dask_image.imread.imread(path_to_image)
sdata['image'] = spatialdata.models.Image2DModel.parse(image)

INFO     no axes information specified in the object, setting `dims` to: ('c', 'y', 'x')                           


In [5]:
#parse mask
mask = dask_image.imread.imread(path_to_mask)
mask = da.squeeze(mask) #remove singleton dimension
sdata['mask'] = spatialdata.models.Labels2DModel.parse(mask)

INFO     no axes information specified in the object, setting `dims` to: ('y', 'x')                                


In [6]:
#parse quantification table
adata = anndata_utils.read_quant(path_to_quant) #create adata from csv
sdata["imaging"] = spatialdata.models.TableModel.parse(adata)

2025-02-04 07:30:00.515 | INFO     | anndata_utils:read_quant:16 -  ---- read_quant : version number 1.1.0 ----
2025-02-04 07:30:00.597 | INFO     | anndata_utils:read_quant:36 -  16808 cells and 15 variables
2025-02-04 07:30:00.598 | INFO     | anndata_utils:read_quant:37 -  ---- read_quant is done, took 0s  ----


In [10]:
sdata

SpatialData object
├── Images
│     └── 'image': DataArray[cyx] (15, 5000, 5000)
├── Labels
│     └── 'mask': DataArray[yx] (5000, 5000)
└── Tables
      └── 'imaging': AnnData (16808, 15)
with coordinate systems:
    ▸ 'global', with elements:
        image (Images), mask (Labels)

In [11]:
#write as zarr object
sdata.write("../data/TNBC_subset/sdata.zarr")

INFO     The Zarr backing store has been changed from None the new file path: ../data/TNBC_subset/sdata.zarr       


In [12]:
# QC1 Visual check of stitching and segmentation
interactive = napari_spatialdata.Interactive(sdata)
interactive.run()

In [13]:
# size considerations:
# if image too big you have to chunk it to save it as a zarr
# if image too big you have to create a downsampled version of the image